# Project Luther

In [507]:
# import modules

import urllib2
from bs4 import BeautifulSoup
import pickle
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import scipy as sp

In [2]:
# load list of box office mojo targets for movies

with open ("movie_list2.pkl") as f:
    movie_list = pickle.load(f)

In [4]:
countries = ['Argentina', 'Australia', 'Austria', 'Belgium', 'Bolivia', 'Brazil', 'Bulgaria', 'Chile', 'China',
             'Colombia', 'Croatia', 'Czech Republic', 'Denmark' , 'East Africa', 'Egypt', 'Estonia', 'Finland',
             'France', 'Germany', 'Ghana', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Italy', 'Japan',
            'Latvia', 'Lebanon', 'Lithuania', 'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Nigeria',
            'Norway', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Romania', 'Russia - CIS', 'Serbia & Montenegro',
            'Singapore', 'Slovakia', 'Slovenia', 'South Africa (Entire Region)', 'South Korea', 'Spain', 'Sweden',
            'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'Ukraine', 'United Arab Emirates', 'United Kingdom',
            'Uruguay', 'Venezuela']

In [5]:
# find domestic gross, foreign gross, country gross for each movie

intl_gross = {}
for movie in movie_list:
    intl_gross.setdefault(movie[:-4], {})

def get_data():
    for k, v in intl_gross.iteritems():
        try:
            url = "http://www.boxofficemojo.com/movies/?page=intl&id=" + k + ".htm"
            page = urllib2.urlopen(url)
            soup = BeautifulSoup(page)
        except: 
            print "Can't scrape "
            continue
        
        # domestic total
        
        domestic_total = soup.find(text = re.compile("Domestic Total"))
        try:
            v['Domestic Total'] = int(str(domestic_total.parent.find("b").string).replace(",", "").replace("$", ""))
        except:
            pass
        
        # foreign total
        
        foreign_total = soup.find(text = "FOREIGN TOTAL")
        try:
            foreign_total = foreign_total.parent.parent.parent.parent.parent.find_all("b")[1]
            try: 
                v['Foreign Total'] = int(str(foreign_total.string).replace(",", "").replace("$", ""))
            except:
                pass
        except:
            pass
        
        # gross for each country
        
        for country in countries:
            foreign_total = soup.find(text = country)
            if foreign_total != None:
                try:
                    v[country] = int(str(foreign_total.parent.parent.parent.parent.parent.find_all("b")[1].string).replace(",", ""). replace("$", ""))
                except:
                    pass

In [6]:
with open("foreign_dict_dan.pkl", "r") as f:
    movies_dict = pickle.load(f)

In [7]:
def get_data_main():
    for k, v in movies_dict.iteritems():
        try:
            url = "http://www.boxofficemojo.com/movies/?page=main&id=" + k + ".htm"
            page = urllib2.urlopen(url)
            soup = BeautifulSoup(page)
        except: 
            print "Can't scrape " + k
            continue
        try:
            v['Genre'] = str(soup.find(text = "Genre: ").next_sibling.string)
        except:
            pass
        try:
            v['Release Date'] = str(soup.find(text = "Release Date: ").next_sibling.string)
        except:
            pass
        try:
            v['Director'] = str(soup.find(text = "Director:").parent.parent.parent.parent.next_sibling.string)
        except:
            pass
        try:
            v['Writer'] = str(soup.find(text = "Writer:").parent.parent.parent.parent.next_sibling.string)
        except:
            pass
        try:
            v['Actors'] = [str(string) for string in list(soup.find(text = "Actors:").parent.parent.parent.parent.next_sibling.strings)]
        except:
            pass
        try:
            v['Producers'] = [str(string) for string in list(soup.find(text = "Producers:").parent.parent.parent.parent.next_sibling.strings)]
        except:
            pass

In [8]:

get_data_main()

elapsed = datetime.datetime.now() - before
print elapsed

Can't scrape likefatherlikeson
Can't scrape spectacularnow
Can't scrape lordoftheflies
Can't scrape sheshavingababy
Can't scrape bodyofevidence
Can't scrape omen2
Can't scrape omen3
Can't scrape rumbleinthebronx
0:25:37.398967


In [506]:
with open("final_scraped_product.pkl", "w") as f:
    pickle.dump(movies_dict, f)

In [204]:
df = pd.DataFrame(movies_dict)
df = df.transpose()

In [205]:
def format_date(n):
    try:
        return datetime.datetime.strptime(n, "%B %d, %Y")
    except:
        try:
            return datetime.datetime.strptime(n, "%B %Y")
        except:
            try:
                return datetime.datetime.strptime(n, "%Y")
            except:
                return np.nan

df["Release Date"] = map(lambda x: format_date(x), df["Release Date"])
df = df.sort("Release Date")

In [206]:
def get_year(dt):
    if type(dt) == datetime.datetime:
        return int(dt.year)
    else:
        return np.nan
df["Release Year"] = df["Release Date"].apply(lambda x: get_year(x))

In [285]:
plt.bar(df["Release Year"], df["Foreign Total"])
plt.xlim([1960,2015])
plt.xlabel("Year of Release")
plt.ylabel("Foreign Total Gross")
plt.title("Foreign Total Gross by Year of Release")
plt.show()

In [462]:
# Separate out movies with non-null Domestic and Foreign totals

dom_for = df[df["Domestic Total"].notnull() & df["Foreign Total"].notnull()]

# create a polyfit 

p = np.polyfit(dom_for["Domestic Total ** 1.5"].astype(float),dom_for["Foreign Total"].astype(float),1)
fit = np.poly1d(p)
print p

[  8.23194784e-05   1.64086033e+07]


In [491]:
plt.scatter(df["Domestic Total ** 1.5"], df["Foreign Total"])
plt.plot(df["Domestic Total ** 1.5"], fit(df["Domestic Total ** 1.5"]))
plt.xlabel("Domestic Total ^ 1.5")
plt.ylabel("Foreign Total")
plt.title("Foreign Total Gross vs Domestic Total Gross ^ 1.5")
plt.show()

In [442]:
df["Genre"].apply(lambda x: x.lower())

fourhorsemen                 war drama
metropolis27                    sci-fi
snowwhite                    animation
wizardofoz                     fantasy
gonewiththewind        historical epic
pinocchio                    animation
fantasia                     animation
citizenkane                      drama
sergeantyork                       war
bambi                        animation
casablanca                     romance
bellsofstmarys                   drama
bestyearsofourlives              drama
duelinthesun                   western
greatestshowonearth              drama
...
thelongestride           romance
monkeykingdom        documentary
cybernatural              horror
truestory                  drama
paulblart2                comedy
bodyofevidence               nan
elizabeth                    nan
likefatherlikeson            nan
lordoftheflies               nan
omen2                        nan
omen3                        nan
rumbleinthebronx             nan
sheshavingababy

In [344]:
df["Genre"] = df["Genre"].astype(str)

In [354]:
def action(x):
    if "action" in x.lower():
        return 1
    else:
        return 0

df["Action"] = df["Genre"].apply(lambda x: action(x))

In [384]:
def comedy(x):
    if "comedy" in x.lower():
        return 1
    else:
        return 0

df["Comedy"] = df["Genre"].apply(lambda x: comedy(x))

In [385]:
def adventure(x):
    if "adventure" in x.lower():
        return 1
    else:
        return 0

df["Adventure"] = df["Genre"].apply(lambda x: adventure(x))

In [386]:
def drama(x):
    if "drama" in x.lower():
        return 1
    else:
        return 0

df["Drama"] = df["Genre"].apply(lambda x: drama(x))

In [347]:
a = "action man"
"action" in a

True

In [389]:
sum(df["Action"])

792

In [357]:
df["Genre"].value_counts()

Comedy             834
Unknown            625
Drama              492
Foreign            334
Romantic Comedy    308
Horror             287
Thriller           259
Animation          254
Action             229
Comedy / Drama     224
Romance            171
Family Comedy      150
Documentary        144
Action Comedy      141
Period Drama       133
...
Historical Drama      23
Sci-Fi                22
Foreign / Action      20
Adventure Comedy      17
Period Action         17
Historical Epic       14
Sports Action         11
Action Fantasy        11
Period Horror         11
Sci-Fi Fantasy        11
Western Comedy        10
nan                   10
Foreign / Horror       6
Romantic Adventure     4
War Romance            4
Length: 66, dtype: int64

In [429]:
com_tot = np.nansum(df[df["Comedy"] == 1]["Foreign Total"].astype(float))
act_tot = np.nansum(df[df["Action"] == 1]["Foreign Total"].astype(float))
drama_tot = np.nansum(df[df["Drama"] == 1]["Foreign Total"].astype(float))
adv_tot = np.nansum(df[df["Adventure"] == 1]["Foreign Total"].astype(float))

genre_df = pd.DataFrame([com_tot, act_tot, drama_tot, adv_tot], index = ["Comedy", "Action", "Drama", "Adventure"])

In [397]:
genre_df.plot(kind="bar", legend = 0)
plt.show()

In [430]:
action_movies = df[df["Action"] == 1]

In [431]:
len(action_movies)

792

In [494]:
plt.scatter(action_movies["Domestic Total"], action_movies["Foreign Total"])
plt.plot(action_movies["Domestic Total"], fit(action_movies["Domestic Total"]))
plt.xlabel("Domestic Total")
plt.ylabel("Foreign Total")
plt.title("Foreign Total vs Domestic Total for Action Movies")
plt.show()

In [493]:
# Separate out movies with non-null Domestic and Foreign totals for action movies

action_dom_for = action_movies[action_movies["Domestic Total"].notnull() & action_movies["Foreign Total"].notnull()]

# create a polyfit 

action_p = np.polyfit(action_dom_for["Domestic Total"].astype(float),action_dom_for["Foreign Total"].astype(float),1)
fit = np.poly1d(action_p)
print p

[  8.23194784e-05   1.64086033e+07]


In [496]:
import scipy
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(action_movies["Domestic Total"], action_movies["Foreign Total"])


In [518]:
country_total = df.apply(np.nansum)

TypeError: ("unsupported operand type(s) for +: 'float' and 'list'", u'occurred at index Actors')

In [554]:
# Separate out movies with non-null Domestic and Foreign totals for action movies

germanydf = df[df["Action"].notnull() & df["China"].notnull()]

# create a polyfit 

germany_p = np.polyfit(germanydf["Action"].astype(float),germanydf["China"].astype(float),1)
fit = np.poly1d(germany_p)
print germany_p

[ 16046724.19374579  16948322.47567568]


In [555]:
plt.scatter(df["Action"], df["China"])
plt.plot(df["Action"], fit(df["Action"]))
plt.xlabel("Action Movie")
plt.ylabel("Germany")
plt.title("Germany vs Action")
plt.show()

In [525]:
est = smf.

2240

In [530]:
df.columns

Index([u'Actors', u'Argentina', u'Australia', u'Austria', u'Belgium', u'Bolivia', u'Brazil', u'Bulgaria', u'Chile', u'China', u'Colombia', u'Croatia', u'Czech Republic', u'Denmark', u'Director', u'Domestic Total', u'East Africa', u'Egypt', u'Estonia', u'Finland', u'Foreign Total', u'France', u'Genre', u'Germany', u'Ghana', u'Greece', u'Hong Kong', u'Hungary', u'Iceland', u'India', u'Italy', u'Japan', u'Latvia', u'Lebanon', u'Lithuania', u'Malaysia', u'Mexico', u'Netherlands', u'New Zealand', u'Nigeria', u'Norway', u'Peru', u'Philippines', u'Poland', u'Portugal', u'Producers', u'Release Date', u'Romania', u'Russia - CIS', u'Serbia & Montenegro', u'Singapore', u'Slovakia', u'Slovenia', u'South Africa (Entire Region)', u'South Korea', u'Spain', u'Sweden', u'Switzerland', u'Taiwan', u'Thailand', u'Turkey', u'Ukraine', u'United Arab Emirates', u'United Kingdom', u'Uruguay', u'Venezuela', u'Writer', u'Release Year', u'Foreign Pct', u'Domestic Total ** 1.5', u'Action', u'Comedy', u'Adventure'

In [538]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [544]:
df